<a href="https://colab.research.google.com/github/jobrosier-vu/Teaching-Notebooks/blob/main/AdvancedSpatialAnalysis/Spatial_Assignment6/Gee_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine (GEE) with Python

In the following example we are working with the Google Earth Engine (GEE) using the Python API. 
The goal of this example is to familiarize you with some of the data that can be found on the GEE and how to handle it.
Some parts of the code used will be relativly complicated to understand. This is a concequence of how the data is structured in the GEE. For now it is not important to understand each line of code. The goal here is make sure you understand the general idea behind the operations and get an impression of what is possible with the GEE in combination with the Python API. For examples and tutorials please visit the [examples](https://github.com/giswqs/geemap/tree/master/examples) page

**Before you start**

Before we start let's go over some core principles of the GEE
- Using the API means that you are working on a GEE server. To distinguish between commands you make on your local machine and commands on the server the API requires you to add ee. before a command.   
- Rasters are called Images and a collection of images is called an ImageCollection. Each image contains metadata such as it's crs, resolution, timestamp, etc. To find out more about an image you can print the metadata by using print(image_name.getInfo())   

Make sure to install the following packages in your environment before starting. These packages you will have to install every time you restart this notebook.

In [ ]:
pip install geemap

In [ ]:
# Import the following packages 
import geemap
import ee
import os

In order to work with the GEE we need to authenticate first. Make sure you have a GEE account and copy paste the code after running the next code block.
Note that you can use your VU e-mail for this, as this will give you access. Yet, a gmail account will also work.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

At this stage, you are connected ti GEE and you can effectively use Python to use the methods you can also use in GEE itself, but now via the Python API. Let's have a look.

In this example we are going to visualize elevation and the max summer precipation data in the Netherlands. These datasets would, for instance, be interesting for a floodrisk assesment. To do this we need a precipitation dataset, an elevation dataset, and a dataset that contains boundaries of the Netherlands. To see all available datasets on the GEE go to the [catalog](https://developers.google.com/earth-engine/datasets/catalog) 

In [ ]:
# As elevation we use the SRTM Digital Elevation Data Version 4
elevation = ee.Image('CGIAR/SRTM90_V4')

# To select data only in the Netherlands we use the level0 boundaries.
Boundaries = ee.FeatureCollection('FAO/GAUL/2015/level0')

"""
We choose the ERA5 Monthly Aggregates as precipitation data. From the catalog we can see the images in this collection
have many bands, but we are only interested in the precipitation. Using .select(band_name) we can select the band in each 
image of the collection.

"""
precipitation_dataset = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_BY_HOUR").select('total_precipitation')

We have a timeseries between 1979 and 2020-06, but in this case we are only interested in the maximum summer precipitation of the last 10 years.
An ImageCollection can be filtered between a certain date range using 'filterDate' and a selection of months can be made by specifying a 'calanderRange'. Try different months or date ranges, but try not to select too much data as it will take longer to process.

In [ ]:
# We take months 7 till 9 as summer.
filterMonth = ee.Filter.calendarRange(7, 9, 'month');
precip_range = precipitation_dataset.filterDate('2010-01-01', '2020-01-01').filter(filterMonth)

# From all selected months we want the maximum precipitation (mean/median/min are also possible, try to see the difference)
precip_max = precip_range.max()

To select a specific region we need to know the exact name. We can check the names by visualizing the boundaries on the map and use the map inspector to find the information we need.

In [ ]:
# set our initial map parameters for The Netherlands. This is not required, but saves some zooming in later.
center_lat = 52.36
center_lon = 4.89
zoomlevel=7

# This will generate the basic map, but not visualize it
Map = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# With addLayer we can add datasets to the map
Map.addLayer(Boundaries)

# To view the map simply type Map 
Map


By selecting the Inspector from the tools menu (right top in the map) we can click on a province to get information on the data. We need the name of the feature that contains the names of the countries, and the name of the country (Netherlands) we want to select. (Both datasets are global so make sure to try different countries to see the difference.) 

In [ ]:
"""
Filtering is done with the .filter feature and a filter equation. From the inspector we can see that the feature 
in the Boundaries layer related to the provinces is called ADM0_NAME, so we use this in the equation.
"""
netherlands = Boundaries.filter(ee.Filter.eq('ADM0_NAME', 'Netherlands'))

With the selected regions we can now cut out our data. 

In [ ]:
# A single image can easily be clipped by using .clip() 
precip_max_NL = precip_max.clip(netherlands.geometry())

elevation_NL = elevation.clip(netherlands.geometry())

In [ ]:
# In order to see the variation on the map we specify a min and max value for visualization. 
# Use the inspector tool to see if these are
# chosen well.
vis_precip = {
    'min': 4,
    'max': 7,
    'palette': ['deebf7', '9ecae1', '3182bd']
}

#The data was in meter, but we want it in mm so we multiply by 1000
Map.addLayer(precip_max_NL.multiply(ee.Number(1000)), vis_precip)

Note that the map above is automatically updated.

In [ ]:
# Let's have another try with the elevation data of the Netherlands
vis_elv = {
    'min': -10,
    'max': 150,
    'palette': ['006633','E5FFCC','662A00','D8D8D8','F5F5F5']
}

Map.addLayer(elevation_NL, vis_elv)
# use the layer button to turn of/on certain layers. this button is included in the toolbar

In order to continue working with the data we can download the data.
To export an image we need to provide at least the following things:
- Region
- scale (resolution)
- filename
- crs

We can export data in different ways:
- The Earth engine way is to export to your Google drive, from which you can download it to your PC.
- The second way is to use the geemap export. By first defining an ouput directory and filename the data is directly dowloaded to your PC.

In [ ]:
"""
# Export to drive
# You can remove the quotation marks to actually use this option. 
# check at https://code.earthengine.google.com/ the status of the download
task = ee.batch.Export.image.toDrive(image=precip_max_NL,
                                     description='Precip_NL',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     folder="ASA",
                                     region=netherlands.geometry(),
                                     scale=27784,
                                     fileNamePrefix='Precip_NL',
                                     maxPixels= 3784216672400,)
task.start()


"""

# Export to the Google Colab Files folder

filename = os.path.join('Precip_NL.tif')
geemap.ee_export_image(precip_max_NL, filename=filename, scale=27784, region=netherlands.geometry(), file_per_band=False)
